<a href="https://colab.research.google.com/github/Fiarma5233/Data-Science/blob/master/ETL_Pratice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Chemin du répertoire dans lequel se trouve votre script Python
directory_path = "/content/drive/MyDrive/ETL_Pratice/"

# Changer de répertoire vers le répertoire spécifié
os.chdir(directory_path)

# Maintenant vous êtes dans le bon répertoire, vous pouvez exécuter votre script ici
# Par exemple, si votre script s'appelle 'mon_script.py', vous pouvez l'exécuter comme ceci :
# python mon_script.py



In [2]:
pwd

'/content/drive/MyDrive/ETL_Pratice'

In [7]:


import glob
#import getroot
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime

# chemins d'acces ayant une portee globale pour les autres fonctions

log_file = "log_file.txt" # fichier de journalisation
target_file = "transformed.csv" # variables ou les donnees finales transformees seront stockees

###############  Extraction   #########################

# fonction pour extraire les donnees de fichiers csv

def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

# fonction pour extraire les donnees de fichiers json
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

# fonction pour extraire les donnees de fichiers xml
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model', 'year_of_manufacture', 'price', 'fuel'])
    # creation d'un dataframe vide

    tree = ET.parse(file_to_process)
    # analyse du fichier xml passer en parametre et cree un objet 'ElementTree' qui represente la structure du fichier xml

    root = tree.getroot()
    # recupere l'element racine de l'arborescence xml (le noeud de plus haut niveau )

    for car in root:
        # boucle a travers chaque element  enfant de la racine (chaque "car" dans le fichier xml)

        car_model = car.find('car_model').text
        # trouve l'ement "car_model" dans l'element "car" actuel et recupere son texte (le modele de la voiture)

        year_of_manufacture = car.find('year_of_manufacture').text
        # trouve l'element "year_of_manufacture" dans l'element "car" actuel et recupere son texte (l'annee de fabrication de la voiture)

        price = float(car.find('price').text)
        # trouve l'element "price" dans l'element "car" actuel et recupere son texte (le prix de la voiture)

        fuel = car.find('fuel').text
        # trouve l'element "fuel" dans l'element "car" actuel et recupere son texte (le carburant de la voiture)

        dataframe = pd.concat([dataframe, pd.DataFrame([{'car_model':car_model, 'year_of_manufacture':year_of_manufacture, 'price':price, 'fuel':fuel}])], ignore_index=True)
        # concatene le dataframe vide avec le nouveau datefame en y ajoutant les valeurs et en ignorant les index

    return dataframe
    # return le dataframe final

    ### fonction pour identifier chaque fonction d'extraction ###

def extract():
    extracted_data = pd.DataFrame(columns=['car_model', 'year_of_manufacture', 'price', 'fuel'])
    # creation d'un dataframe vide

     # traitement de tous les fichiers csv
    for csvfile in glob.glob("*.csv"):
        # boucle a travers tous les fichiers csv dans le repertoire courant

        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_csv(csvfile))], ignore_index=True)
        # Utilise la fonction 'extract_from_csv' pour extraire les données du fichier CSV actuel,
        # convertit les données extraites en DataFrame, et les concatène au DataFrame 'extracted_data',
        # en ignorant l'index pour éviter des indices dupliqués.

     # traitement de tous les fichiers json
    for jsonfile in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_json(jsonfile))], ignore_index=True)

    # traitement de tous les fichiers xml
    for xmlfile in glob.glob("*.xml"):
        extracted_data = pd.concat([extracted_data, pd.DataFrame(extract_from_xml(xmlfile))], ignore_index=True)

    return extracted_data
    # return les donnees extraites


################# Transformation ###############

# ici il faut transformer le prix des voitures en decimal avec 2 chiffres apres la virgule

def transform(data):

    data['price'] = round(data.price * 1.00000000,2)

    return data
    # return data


 ################# Load ###############


def load_data(target_file, transformed_data):
     # Déclare une fonction nommée 'load_data' qui prend deux paramètres :
    # 'target_file' : le nom du fichier cible où les données transformées seront sauvegardées.
    # 'transformed_data' : le DataFrame contenant les données transformées.


    transformed_data.to_csv(target_file)
    # Sauvegarde le DataFrame 'transformed_data' dans un fichier CSV dont le chemin est spécifié par 'target_file'.
    # La méthode 'to_csv' de pandas est utilisée pour écrire le contenu du DataFrame dans un fichier CSV.


 ################# Logging ###############

def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    # Définit le format de l'horodatage. Le format inclut l'année, le nom du mois, le jour, l'heure, les minutes et les secondes.

    now = datetime.now()
    # Recupere l'horodatage actuel

    timestamp = now.strftime(timestamp_format)
    # formate l'horodatage actuel en une chaine de caractere selon le format de horodatage definit

    with open('log_file', 'a')  as f:
        # Ouvre le fichier de journalisation (log_file) en mode ajout ("a").
        # Le mode "a" permet d'ajouter des données à la fin du fichier sans écraser son contenu.


        f.write(timestamp + "," + message + "\n")
        # Écrit dans le fichier de journalisation l'horodatage suivi d'une virgule, du message, et d'un saut de ligne.


############## Test des operations ETL et les traitements de journalisation ##############


log_progress(" Processus d'Extraction Transformation Chargement de Donnees")

log_progress(" Debut d'extraction des donnees")
extracted_data = extract()
log_progress(" Fin d'extraction des donnees")

log_progress(" Debut de transformation des donnees")
transformed_data = transform(extracted_data)
print("Donnees transformees")
print(transformed_data)


log_progress(" Debut de chargement  des donnees")
load_data(target_file, transformed_data)
log_progress(" Fin de chargement des donnees")



log_progress("Fin du Processus d'Extraction Transformation Chargement de Donnees")




Donnees transformees
         car_model year_of_manufacture     price    fuel  Unnamed: 0.1  \
0             ritz                2014   5000.00  Petrol           NaN   
1              sx4                2013   7089.55  Diesel           NaN   
2             ciaz                2017  10820.90  Petrol           NaN   
3          wagon r                2011   4253.73  Petrol           NaN   
4            swift                2014   6865.67  Diesel           NaN   
..             ...                 ...       ...     ...           ...   
265          camry                2006   3731.34  Petrol           NaN   
266   land cruiser                2010  52238.81  Diesel           NaN   
267  corolla altis                2012   8805.97  Petrol           NaN   
268     etios liva                2013   5149.25  Petrol           NaN   
269        etios g                2014   7089.55  Petrol           NaN   

     Unnamed: 0  
0           NaN  
1           NaN  
2           NaN  
3           NaN  


In [4]:
transformed_data.describe()

,price,Unnamed: 0
count,180.000000,90.000000
mean,10739.800667,44.500000
std,9872.052178,26.124701
min,522.390000,0.000000
25%,4477.610000,22.250000
50%,7201.490000,44.500000
75%,11194.030000,66.750000
max,52238.810000,89.000000


In [5]:
transformed_data.isnull()

,car_model,year_of_manufacture,price,fuel,Unnamed: 0
0,False,False,False,False,True
1,False,False,False,False,True
2,False,False,False,False,True
3,False,False,False,False,True
4,False,False,False,False,True
...,...,...,...,...,...
175,False,False,False,False,True
176,False,False,False,False,True
177,False,False,False,False,True
178,False,False,False,False,True
